In [1]:
data_dir='/home/ubuntu/intelliscope/pcg_data/'
model_dir='/home/ubuntu/intelliscope/notebooks/physionet2016/pcg/models/'

In [2]:
PARAMS = {
          'epochs' : 15,
         'validation_split':0.15,
         'batch_size' : 32,
            'model': 'complex',
          'data_balanced':False,
            'num_epochs':15,
            'opt':'adam',
            'tags': ['instance_norm added', 'dropout added'],
         }

In [3]:
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from keras import backend as K
import tensorflow_addons as tfa


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf

Using TensorFlow backend.


In [4]:


#k1,k2=100,100
k1,k2=272, 462

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(4, kernel_size=(7,7), padding='same', activation='relu', input_shape=(272,462, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(3,3)),
    tfa.layers.InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(16, kernel_size=(7,7), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3,3)),
    tfa.layers.InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, kernel_size=(7,7), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3,3)),
    tfa.layers.InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    tfa.layers.InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=PARAMS['opt'],
              metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Recall(),keras.metrics.Precision()])

In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size=PARAMS['batch_size']

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        data_dir+'pcg_other_train_imb/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

validation_generator = val_datagen.flow_from_directory(
        data_dir+'pcg_other_val_imb/',
        target_size=(k1, k2),
        batch_size=batch_size,
        class_mode='categorical')

history=model.fit_generator(
        train_generator,
        #steps_per_epoch=2000 // batch_size,
        epochs=PARAMS['num_epochs'],
        validation_data=validation_generator,
        #validation_steps=800 // batch_size
)



/home/ubuntu/core-env/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Found 11359 images belonging to 2 classes.
Found 2433 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 355 steps, validate for 77 steps
Epoch 1/15
355/355 [==============================] - 702s 2s/step - loss: 0.2562 - categorical_accuracy: 0.9068 - recall: 0.9068 - precision: 0.9068 - val_loss: 0.7320 - val_categorical_accuracy: 0.7201 - val_recall: 0.7201 - val_precision: 0.7201
Epoch 2/15
355/355 [==============================] - 691s 2s/step - loss: 0.1384 - categorical_accuracy: 0.9439 - recall: 0.9439 - precision: 0.9439 - val_loss: 0.8420 - val_categorical_accuracy: 0.6963 - val_recall: 0.6963 - val_precision: 0.6963
Epoch 3/15
355/355 [==============================] - 692s 2s/step - loss: 0.1054 - categorical_accuracy: 0.9527 - recall: 0.9527 - precision: 0.9527 - val_loss: 1.2174 - val_categorical_accuracy: 0.6732 - val_recall: 0.6732 - val_precision: 0.6732

In [6]:

#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
os.environ['NEPTUNE_PROJECT']="icassp/PCG"
#os.environ['NEPTUNE_NOTEBOOK_ID']="4897a223-8a9f-4f00-8cec-22eda690858d"
os.environ['NEPTUNE_NOTEBOOK_PATH']='PCG-complex (imbalanced).ipynb'

neptune.init('icassp/PCG',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(description='PCG-complex (imbalanced)',params=PARAMS,upload_source_files=['PCG-complex (imbalanced).ipynb'],tags=PARAMS['tags'],upload_stdout=True)

model.save(f'{model_dir}{str(exp)}.h5')
neptune.log_artifact(f'{model_dir}{str(exp)}.h5')

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/icassp/PCG/e/PCG-55


In [ ]:
no=1
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision_%d'%no][i])
    neptune.log_metric('val_precision',history.history['val_precision_%d'%no][i])
    neptune.log_metric('recall',history.history['recall_%d'%no][i])
    neptune.log_metric('val_recall',history.history['val_recall_%d'%no][i])

In [7]:
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision'][i])
    neptune.log_metric('val_precision',history.history['val_precision'][i])
    neptune.log_metric('recall',history.history['recall'][i])
    neptune.log_metric('val_recall',history.history['val_recall'][i])

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_dir+'pcg_other_test_imb/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=len(os.listdir(data_dir+'pcg_other_test_imb/1'))+len(os.listdir(data_dir+'pcg_other_test_imb/0')),
        class_mode='categorical')

X,y = test_generator.next()
prediction = model.predict(X)

import numpy as np

y_test=np.argmax(y,axis=1)
y_pred=prediction
from neptunecontrib.monitoring.metrics import *

threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_brier_loss(y_test, y_pred[:, 1])
log_log_loss(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)
log_prediction_distribution(y_test, y_pred[:, 1])


/home/ubuntu/core-env/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Found 2434 images belonging to 2 classes.


In [9]:
neptune.stop()